# Hyper parameter tunning
- updated 0922
- data  
7th_FE_ver4  
np.log1p()
np.expm1()

catboost 튜닝
- optuna, bayesian

## y_predict 값들 피클로 저장해놓기   
## ISSUE : get dummies 이용
stacking & scaler 구현


## ready

In [ ]:
cd /content/drive/My Drive/빅콘

/content/drive/My Drive/빅콘


In [ ]:
!apt-get update --qq
!apt-get install fonts-nanum* -qq

E: Command line option --qq is not understood in combination with the other options
Selecting previously unselected package fonts-nanum.
(Reading database ... 144676 files and directories currently installed.)
Preparing to unpack .../fonts-nanum_20170925-1_all.deb ...
Unpacking fonts-nanum (20170925-1) ...
Selecting previously unselected package fonts-nanum-eco.
Preparing to unpack .../fonts-nanum-eco_1.000-6_all.deb ...
Unpacking fonts-nanum-eco (1.000-6) ...
Selecting previously unselected package fonts-nanum-extra.
Preparing to unpack .../fonts-nanum-extra_20170925-1_all.deb ...
Unpacking fonts-nanum-extra (20170925-1) ...
Selecting previously unselected package fonts-nanum-coding.
Preparing to unpack .../fonts-nanum-coding_2.5-1_all.deb ...
Unpacking fonts-nanum-coding (2.5-1) ...
Setting up fonts-nanum-extra (20170925-1) ...
Setting up fonts-nanum (20170925-1) ...
Setting up fonts-nanum-coding (2.5-1) ...
Setting up fonts-nanum-eco (1.000-6) ...
Processing triggers for fontconfig 

In [ ]:
# 한글 폰트
import matplotlib.font_manager as fm
fm._rebuild()

In [ ]:
# # CUDA
# !pip install mxnet-cu80
# import mxnet as mx

In [ ]:
# ready
import os
import sys
import joblib
import pickle
import warnings
warnings.filterwarnings(action='ignore')

import pandas as pd 
import numpy as np
import random
import math

from IPython.display import display
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
pd.set_option('display.max_info_columns', 500)

import imp
import datetime
from tqdm import tqdm

In [ ]:
# vusualize
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use('seaborn')
plt.rc('font', family='NanumGothic')
plt.rc('axes', unicode_minus=False)

In [ ]:
# sklearn
from sklearn.model_selection import KFold, cross_val_score, train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.metrics import accuracy_score

In [ ]:
!pip install shap
import shap

     |████████████████████████████████| 327kB 8.0MB/s 
  Created wheel for shap: filename=shap-0.36.0-cp36-cp36m-linux_x86_64.whl size=456479 sha256=8c2b6653f22e12a741e14135455260363f3063c9838e34c4bfd14a8a7587590d
  Stored in directory: /root/.cache/pip/wheels/fb/15/e1/8f61106790da27e0765aaa6e664550ca2c50ea339099e799f4
Successfully built shap


### Load data

In [ ]:
# load data
data = joblib.load(os.path.join('data', '7th_train_FE.pkl'))
locals().update(data)

# X, y 설정
X = data['X'] ; y = data['y'] ; log_y = np.log1p(y)

In [ ]:
X.head()

,노출(분),판매단가,NEW_최고판매단가,NEW_최저판매단가,NEW_중간판매단가,NEW_최고-최저,상품군_최고판매단가,상품군_최저판매단가,상품군_중간판매단가,상품군_최고-최저,마더코드_최고판매단가,마더코드_최저판매단가,마더코드_최고-최저,방송시간차,조기매진,옵션여부,isHoliday,방송월,방송일,평일여부,상품군평균판매단가차이,결합상품,상품군_zscore,마더코드_zscore,NEW_zscore,상품군별월별평균판매량,상품군별시간대별평균판매량,상품군별시간분별평균판매량,할인율,ema_s,ema_m,ema_l,macd,sig,rol14,rol30,v0,v1,v2,v3,v4,v5,v6,v7,v8,v9,v10,v11,v12,v13,v14,v15,v16,v17,v18,v19,v20,v21,v22,v23,v24,v25,v26,v27,v28,v29,v30,v31,v32,v33,v34,v35,v36,v37,v38,v39,v40,v41,v42,v43,v44,v45,v46,v47,v48,v49,v50,v51,v52,v53,v54,v55,v56,v57,v58,v59,v60,v61,v62,v63,v64,v65,v66,v67,v68,v69,v70,v71,v72,v73,v74,v75,v76,v77,v78,v79,v80,v81,v82,v83,v84,v85,v86,v87,v88,v89,v90,v91,v92,v93,v94,v95,v96,v97,v98,v99,v100,v101,v102,v103,v104,v105,v106,v107,v108,v109,합계,컴퓨터 및 주변기기,가전·전자·통신기기,서적,사무·문구,의복,신발,가방,패션용품 및 액세서리,스포츠·레저용품,화장품,아동·유아용품,음·식료품,농축수산물,생활용품,자동차 및 자동차용품,가구,애완용품,여행 및 교통서비스,문화 및 레저서비스,e쿠폰서비스,음식서비스,기타서비스,기타,현재생활형편CSI,현재경기판단CSI,생활형편전망CSI,소비지출전망CSI,주택가격전망CSI,임금수준전망CSI,소비자심리지수,경상지수,불변지수,pca_1,pca_2,pca_3,pca_4,pca_5,강수량(mm)_경기,강수량(mm)_광주,강수량(mm)_대구,강수량(mm)_대전,강수량(mm)_부산,강수량(mm)_서울,강수량(mm)_울산,강수량(mm)_인천,기온(°C)_경기,기온(°C)_광주,기온(°C)_대구,기온(°C)_대전,기온(°C)_부산,기온(°C)_서울,기온(°C)_울산,기온(°C)_인천,습도(%)_경기,습도(%)_광주,습도(%)_대구,습도(%)_대전,습도(%)_부산,습도(%)_서울,습도(%)_울산,습도(%)_인천,시정(10m)_경기,시정(10m)_광주,시정(10m)_대구,시정(10m)_대전,시정(10m)_부산,시정(10m)_서울,시정(10m)_울산,시정(10m)_인천,지면온도(°C)_경기,지면온도(°C)_광주,지면온도(°C)_대구,지면온도(°C)_대전,지면온도(°C)_부산,지면온도(°C)_서울,지면온도(°C)_울산,지면온도(°C)_인천,체감온도_경기,체감온도_광주,체감온도_대구,체감온도_대전,체감온도_부산,체감온도_서울,체감온도_울산,체감온도_인천,풍속(m/s)_경기,풍속(m/s)_광주,풍속(m/s)_대구,풍속(m/s)_대전,풍속(m/s)_부산,풍속(m/s)_서울,풍속(m/s)_울산,풍속(m/s)_인천,최고PM10_경기,최고PM10_광주,최고PM10_대구,최고PM10_대전,최고PM10_부산,최고PM10_서울,최고PM10_울산,최고PM10_인천,최고PM25_경기,최고PM25_광주,최고PM25_대구,최고PM25_대전,최고PM25_부산,최고PM25_서울,최고PM25_울산,최고PM25_인천,최저PM10_경기,최저PM10_광주,최저PM10_대구,최저PM10_대전,최저PM10_부산,최저PM10_서울,최저PM10_울산,최저PM10_인천,최저PM25_경기,최저PM25_광주,최저PM25_대구,최저PM25_대전,최저PM25_부산,최저PM25_서울,최저PM25_울산,최저PM25_인천,평균PM10_경기,평균PM10_광주,평균PM10_대구,평균PM10_대전,평균PM10_부산,평균PM10_서울,평균PM10_울산,평균PM10_인천,평균PM25_경기,평균PM25_광주,평균PM25_대구,평균PM25_대전,평균PM25_부산,평균PM25_서울,평균PM25_울산,평균PM25_인천,상품군_가구,상품군_가전,상품군_건강기능,상품군_농수축,상품군_생활용품,상품군_속옷,상품군_의류,상품군_이미용,상품군_잡화,상품군_주방,상품군_침구,결제방법_0.0,결제방법_1.0,결제방법_2.0,성별_0.0,성별_1.0,성별_2.0,NS카테고리_TV/냉장고/세탁기,NS카테고리_가방/패션잡화,NS카테고리_간편/즉석요리,NS카테고리_거실가구,NS카테고리_건강/안마/실버,NS카테고리_건강기능,NS카테고리_계절가전,NS카테고리_과자/간식,NS카테고리_김치/반찬,NS카테고리_남성의류,NS카테고리_노트북/PC/태블릿/주변기기,NS카테고리_농산,NS카테고리_등산/캠핑/낚시/자전거,NS카테고리_라면/면/스프,NS카테고리_블랙박스/자동차,NS카테고리_생활가전,NS카테고리_생활잡화,NS카테고리_세제/욕실/위생,NS카테고리_세탁/청소용품,NS카테고리_수납/정리용품,NS카테고리_수납가구,NS카테고리_수산,NS카테고리_수입명품,NS카테고리_스포츠의류/슈즈/잡화,NS카테고리_시공/DIY,NS카테고리_언더웨어,NS카테고리_여성의류,NS카테고리_영상/음향가전,NS카테고리_자동차용품,NS카테고리_전통건강,NS카테고리_조미료/소스/장류,NS카테고리_주방가전,NS카테고리_주방용품/식기,NS카테고리_쥬얼리/시계,NS카테고리_축산,NS카테고리_침구/커튼/카페트,NS카테고리_침실가구,NS카테고리_캐주얼/유니섹스,NS카테고리_커피/생수/음료,NS카테고리_탕/국/찌개,NS카테고리_패션슈즈,NS카테고리_펫용품,NS카테고리_헤어/바디,NS카테고리_헬스/요가/수영/스키,NS카테고리_홈인테리어,NS카테고리_화장품/향수,전체_가격대_고가,전체_가격대_저가,전체_가격대_중저가,전체_가격대_초고가,상품군_가격대_0.0,상품군_가격대_1.0,상품군_가격대_2.0,방송시간(시간)_0,방송시간(시간)_1,방송시간(시간)_10,방송시간(시간)_11,방송시간(시간)_12,방송시간(시간)_13,방송시간(시간)_14,방송시간(시간)_15,방송시간(시간)_16,방송시간(시간)_17,방송시간(시간)_18,방송시간(시간)_19,방송시간(시간)_2,방송시간(시간)_20,방송시간(시간)_21,방송시간(시간)_22,방송시간(시간)_23,방송시간(시간)_6,방송시간(시간)_7,방송시간(시간)_8,방송시간(시간)_9,방송시간(분)_0,방송시간(분)_10,방송시간(분)_15,방송시간(분)_20,방송시간(분)_30,방송시간(분)_35,방송시간(분)_40,방송시간(분)_45,방송시간(분)_5,방송시간(분)_50,방송시간대_밤,방송시간대_아침,방송시간대_오전,방송시간대_오후,방송시간대_저녁,계절_가을,계절_겨울,계절_봄,계절_여름,분기_1분기,분기_2분기,분기_3분기,분기_4분기
0,20.0,39900,39900.0,39900.0,39900.0,0.0,2490000,29000,69000,2461000,69900.0,39900.0,30000.0,0.0,0,0,1,1,1,0,115589.239631,0,-0.323839,-1.489722,0.0,311.943548,316.961219,288.507003,0.0,358.458333,358.458333,358.458333,0.0,0.0,NaN,NaN,-0.024982,-0.006339,-0.028377,0.047281,0.020358,-0.012922,-0.037356,-0.031141,-0.037508,-0.011338,0.002462,0.007924,-0.035870,-0.000225,0.044042,0.024233,0.041229,-0.003493,-0.015973,-0.048715,-0.041179,0.016668,0.024836,-0.035335,-0.048956,0.042389,-0.045046,0.046956,-0.045242,0.025934,-0.018409,0.000041,0.041418,0.014477,-0.042081,-0.028921,-0.007286,0.04544,0.019221,0.016016,-0.028846,0.

In [ ]:
# load data ver5
data5 = joblib.load(os.path.join('data', '5th_FE_ver4.pkl'))
locals().update(data)

# X, y 설정
X5 = data5['X']

In [ ]:
# load data ver5
data_label = joblib.load(os.path.join('data', '5th_FE_ver4_before.pkl'))
locals().update(data)

# X, y 설정
X_label = data_label['X']

In [ ]:
# na
X.isnull().sum().sum()

3733

In [ ]:
# 확인
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 35379 entries, 0 to 38303
Data columns (total 402 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   노출(분)                   35379 non-null  float64
 1   판매단가                    35379 non-null  int64  
 2   NEW_최고판매단가              35379 non-null  float64
 3   NEW_최저판매단가              35379 non-null  float64
 4   NEW_중간판매단가              35379 non-null  float64
 5   NEW_최고-최저               35379 non-null  float64
 6   상품군_최고판매단가              35379 non-null  int64  
 7   상품군_최저판매단가              35379 non-null  int64  
 8   상품군_중간판매단가              35379 non-null  int64  
 9   상품군_최고-최저               35379 non-null  int64  
 10  마더코드_최고판매단가             35379 non-null  float64
 11  마더코드_최저판매단가             35379 non-null  float64
 12  마더코드_최고-최저              35379 non-null  float64
 13  방송시간차                   35379 non-null  float64
 14  조기매진                    35379 non-nul

In [ ]:
# fill na
X0 = X.fillna(0)

# One Hot
X_dum = pd.get_dummies(X)
print(X.shape, X_dum.shape)
# 모두 더미화 되어있음

(35379, 402) (35379, 402)


### MAPE + CV function

In [ ]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [ ]:
# mape
def MAPE(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [ ]:
# mape with exp
def MAPE_exp(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((np.expm1(y_true) - np.expm1(y_pred)) / np.expm1(y_true))) * 100

# Stacking + SHAP
#### CV + Save Prediction + SHAP
SHAP 저장하도록 수정 0924

In [ ]:
# stacking with cv=5
# MAPE_exp_cv 구현

def MAPE_exp_st(model, X, y, cv_splits, version, scaling=False):        
    # setting
    mape = {'val_mape' : [], 'test_mape' : [],'final_mape' : []}
    pred = {'val_idx' : [], 'val_pred' : [],
            'test_idx' : [], 'test_pred' : [],
            'final_pred' : []}      # final : test set mean 값
    # SHAP
    SHAP = {'shap_values' : [], 'expected_value' : []}

    # split train, test
    X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, random_state=77)
    pred['test_idx'].append(X_test.index)
    
    # for 문 돌리기
    cv=KFold(n_splits=cv_splits,random_state=77,shuffle=True)
    for t,v in cv.split(X_train):
        X_train_ = X_train.iloc[t]       # 훈련용
        X_val_ = X_train.iloc[v]         # 검증용 분리.
        y_train_ = y_train.iloc[t]
        y_val_ = y_train.iloc[v]

        pred['val_idx'].append(v)

        # scaling : MinMax or Standard
        if scaling : 
            if scaling == 'MinMax':
                scaler = MinMaxScaler()
            elif scaling == 'Standard':
                scaler = StandardScaler()
            # fitting
            X_train_ = scaler.fit_transform(X_train_)
            X_val_ = scaler.transform(X_val_)
            X_test_ = scaler.transform(X_test)
        
        else:
            X_test_ = X_test

        # modeling
        model.fit(X_train_,y_train_)

        # predict
        y_pred = model.predict(X_val_)
        pred['val_pred'].append(np.expm1(y_pred))
        test_pred = model.predict(X_test_)
        pred['test_pred'].append(np.expm1(test_pred))

        # mape
        mape['val_mape'].append(MAPE_exp(y_val_, y_pred))
        mape['test_mape'].append(MAPE_exp(y_test,test_pred))

        #SHAP
        explainer = shap.TreeExplainer(model)
        shap_values = explainer.shap_values(X_test_)

        SHAP['shap_values'].append(shap_values)
        SHAP['expected_value'].append(explainer.expected_value)

    # final values
    final_test = np.mean(pred['test_pred'], axis=0)
    final_mape = np.mean(mape['test_mape'], axis=0)
    pred['final_pred'].append(final_test)
    mape['final_mape'].append(final_mape)
    
    # loss
    #loss = - np.mean(mape)

    # save pickle
    with open('mape_' + version + '.pickle', 'wb') as f:
        pickle.dump(mape, f, pickle.HIGHEST_PROTOCOL)
    with open('pred_' + version + '.pickle', 'wb') as f:
        pickle.dump(pred, f, pickle.HIGHEST_PROTOCOL)
    with open('SHAP_' + version + '.pickle', 'wb') as f:
        pickle.dump(SHAP, f, pickle.HIGHEST_PROTOCOL)
    

    return mape, pred, SHAP

## Tunning CatBoost

### ready catboost

In [ ]:
!pip install catboost
from catboost import CatBoostRegressor

     |████████████████████████████████| 66.1MB 46kB/s 


In [ ]:
!pip install bayesian-optimization
from bayes_opt import BayesianOptimization

  Created wheel for bayesian-optimization: filename=bayesian_optimization-1.2.0-cp36-none-any.whl size=11685 sha256=0b792ad6452a504c71e93952029a487d1d0dedc3b4df0ad206b2cbba326848d5
  Stored in directory: /root/.cache/pip/wheels/5a/56/ae/e0e3c1fc1954dc3ec712e2df547235ed072b448094d8f94aec
Successfully built bayesian-optimization


### CatBoost + Bayesian 1 

In [ ]:
#np.where(X5.dtypes == np.int)
#np.where(X_label.dtypes == 'object')
#X_dum2 = pd.get_dummies(X_label)
#X_dum2.head()
# categorical_features 사용하지 않음
#categorical_features_indices = np.where((X.dtypes != np.float) & (X.dtypes != np.int))[0]
#categorical_features_indices

In [ ]:
# Setting
X, y = X, log_y 
def catboost_eval(bagging_temperature ,    # 3h 30min
                  depth , 
                  learning_rate ,
                  min_data_in_leaf ,  
                  border_count):
    
    n_splits=5
    cv = KFold(n_splits=5, shuffle=True, random_state=77)
    mape = []
    predict = None

    params = {}
    params['iterations'] = 10000
    params['eval_metric'] = 'MAPE'
    params['random_seed'] = 77
    params['learning_rate'] = learning_rate
    params['min_data_in_leaf'] = int(round(min_data_in_leaf))
    params['depth'] = int(round(depth))
    params['border_count'] = int(round(border_count))
    params['bagging_temperature'] = int(round(bagging_temperature))

    for tr_ind, val_ind in cv.split(X , y):
        X_train = X.iloc[tr_ind]
        y_train = y.iloc[tr_ind]
        X_valid = X.iloc[val_ind]
        y_valid = y.iloc[val_ind]

        # scaling
        scaler = MinMaxScaler()
        X_train = scaler.fit_transform(X_train)
        X_valid = scaler.transform(X_valid)
        
        model = CatBoostRegressor(**params , 
                                  leaf_estimation_iterations = 10,
                                  use_best_model=True,
                                  od_type="Iter",
                                  task_type="GPU"
                                )
        model.fit(X_train, 
                  y_train,
                  eval_set=(X_valid, y_valid),
                  #iterations = 10000,
                  verbose = False
                  )
        
        y_pred = model.predict(X_valid)
        mape.append(MAPE_exp(y_pred,y_valid))
    return -np.mean(mape)

catBO = BayesianOptimization(catboost_eval,
                             {'bagging_temperature': (0, 1000),
                              'depth': (5, 8.99) ,
                              "learning_rate" : (0.001,0.1) , 
                              "min_data_in_leaf" : (1,6) , 
                              'border_count': (5, 255) ,
                             },
                             random_state=0)     # 77

catBO.maximize(init_points=10, n_iter=20)


In [ ]:
# best parameter
catBO.max

{'params': {'bagging_temperature': 641.7274273159518,
  'border_count': 39.73514206561576,
  'depth': 8.785912134398988,
  'learning_rate': 0.02017695398268462,
  'min_data_in_leaf': 2.7106426642135535},
 'target': -28.824896598099542}

### best Catboost + SHAP
https://github.com/cryingjin/Shopping_GG/blob/jbeen2/Modeling/0915_Tuning2.ipynb

In [ ]:
'''
best_params = {
    'bagging_temperature': round(641.7274273159518),
    'border_count': round(39.73514206561576),
    'depth': round(8.785912134398988),
    'learning_rate': 0.0202,     #0.02017695398268462
    'min_data_in_leaf': round(2.7106426642135535)}
'''
# best catboost
cb_BO = CatBoostRegressor(
    bagging_temperature = round(641.7274273159518),
    border_count = round(39.73514206561576),
    depth = round(8.785912134398988),
    learning_rate = 0.0202,     #0.02017695398268462
    min_data_in_leaf = round(2.7106426642135535),
    leaf_estimation_iterations = 10,

    loss_function='MAPE',
    od_type="Iter",
    #eval_metric="AUC",
    task_type="GPU",
    iterations=10000,
    verbose=5000,
    )

In [ ]:
#best_cb.virtual_ensembles_predict
cb_BO

In [ ]:
# 평가 및 저장
mape_catBO, pred_catBO, SHAP_catBO = MAPE_exp_st(cb_BO, X, log_y, 5, 'catBO_0922', scaling='MinMax') 
# 2h + a

0:	learn: 0.0488429	total: 107ms	remaining: 17m 45s
5000:	learn: 0.0172931	total: 7m 36s	remaining: 7m 36s
9999:	learn: 0.0170381	total: 15m 29s	remaining: 0us
0:	learn: 0.0486737	total: 108ms	remaining: 18m 1s
5000:	learn: 0.0175891	total: 7m 43s	remaining: 7m 43s
9999:	learn: 0.0172854	total: 15m 35s	remaining: 0us
0:	learn: 0.0488426	total: 104ms	remaining: 17m 18s
5000:	learn: 0.0178003	total: 7m 45s	remaining: 7m 45s
9999:	learn: 0.0174229	total: 15m 40s	remaining: 0us
0:	learn: 0.0486540	total: 107ms	remaining: 17m 48s
5000:	learn: 0.0178197	total: 7m 42s	remaining: 7m 41s
9999:	learn: 0.0174812	total: 15m 31s	remaining: 0us
0:	learn: 0.0488669	total: 103ms	remaining: 17m 13s
5000:	learn: 0.0174321	total: 7m 35s	remaining: 7m 35s
9999:	learn: 0.0171273	total: 15m 17s	remaining: 0us


In [ ]:
mape_catBO

{'final_mape': [43.97830423209804],
 'test_mape': [43.15712847934601,
  43.48934052081531,
  44.47230947992317,
  43.91217366291737,
  44.860569017488366],
 'val_mape': [44.29497182444521,
  42.480774813138716,
  45.825055340301766,
  44.20043306253304,
  47.1510410278104]}

In [ ]:
pred_catBO

{'final_pred': [array([ 7337454.06888649, 12836978.76630708, 22969872.35931302, ...,
         36402360.29188809, 12124641.20711445, 11494854.33233659])],
 'test_idx': [Int64Index([ 8734,  7885,  2413, 12475,  4571,  9835, 35907, 14833, 10316,
              31140,
              ...
              25556, 30587,  6430, 11575, 33365, 14158, 28909, 34363, 26499,
               4615],
             dtype='int64', length=7076)],
 'test_pred': [array([ 7461435.2239652 , 10557717.41728055, 23894949.15386017, ...,
         35513894.20933883, 12120182.22411971, 11519053.78391614]),
  array([ 7601448.90771597, 13921693.12071874, 21934620.88420022, ...,
         39891709.87014625, 13856132.4276284 , 11470092.0453258 ]),
  array([ 7432880.10625217, 11227923.81908253, 22974984.89611312, ...,
         41359930.50105048, 10656494.10370523, 10639620.3791659 ]),
  array([ 7357800.3096538 , 12802896.78378526, 22298655.92877083, ...,
         33276259.39804833, 13573140.7957979 , 11543905.15723657]),
  array

In [ ]:
SHAP_catBO

{'expected_value': [16.576933918962283,
  16.580584109528008,
  16.57420666919765,
  16.577127688019747,
  16.574577619578996],
 'shap_values': [array([[ 4.63245740e-03,  5.87147233e-03,  1.17219703e-02, ...,
           7.41385555e-05, -7.25387629e-05,  2.40674398e-04],
         [ 2.98230024e-03, -9.60202776e-04,  5.75277243e-03, ...,
          -8.03367202e-06,  2.15578149e-04,  5.62376553e-05],
         [ 3.94514954e-03,  1.37590817e-02,  7.04159760e-03, ...,
           1.56692615e-04, -7.74543642e-05, -4.08986016e-04],
         ...,
         [ 9.65387222e-03,  2.24792065e-03,  7.36174504e-03, ...,
           1.53060624e-04,  1.54371323e-04, -7.75945689e-04],
         [ 6.81013197e-03,  7.25782624e-03,  1.34539664e-02, ...,
           2.81199496e-05,  3.98526642e-04, -1.06682736e-03],
         [ 4.24242564e-03,  1.43624443e-03, -6.92165065e-03, ...,
          -1.35605003e-05,  1.55674166e-04, -1.84271647e-03]]),
  array([[-9.76021623e-04, -3.46016304e-03, -1.51178849e-03, ...,
       

### catboost + 거의 defalut


In [ ]:
# best catboost
'''
비교
cb_BO = CatBoostRegressor(
    bagging_temperature = round(641.7274273159518),
    border_count = round(39.73514206561576),
    depth = round(8.785912134398988),
    learning_rate = 0.0202,     #0.02017695398268462
    min_data_in_leaf = round(2.7106426642135535),
    leaf_estimation_iterations = 10,

    loss_function='MAPE',
    od_type="Iter",
    #eval_metric="AUC",
    task_type="GPU",
    iterations=10000,
    verbose=5000,
    )
'''

In [ ]:
cb_basic = CatBoostRegressor(iterations=10000,
                             learning_rate=0.05,
                             depth=10,
                             eval_metric='MAPE',
                             random_seed = 77,
                             bagging_temperature = 0.2,
                             #group_id = []
                             od_type='Iter',
                             metric_period = 50,
                             od_wait=20,
                             thread_count=-1,
                             #task_type='gpu',
                             verbose=5000) 

In [ ]:
# 평가 및 저장
%%time
mape_catBA, pred_catBA, SHAP_catBA = MAPE_exp_st(cb_basic, X, log_y, 5, 'catBA_0922', scaling='MinMax') 

0:	learn: 0.0479218	total: 259ms	remaining: 43m 11s
5000:	learn: 0.0031240	total: 20m 33s	remaining: 20m 32s
9999:	learn: 0.0010111	total: 41m 8s	remaining: 0us
0:	learn: 0.0479028	total: 247ms	remaining: 41m 14s
5000:	learn: 0.0031103	total: 20m	remaining: 19m 59s
9999:	learn: 0.0009072	total: 39m 59s	remaining: 0us
0:	learn: 0.0480379	total: 252ms	remaining: 41m 56s
5000:	learn: 0.0031696	total: 20m 23s	remaining: 20m 22s
9999:	learn: 0.0009879	total: 40m 37s	remaining: 0us
0:	learn: 0.0479014	total: 260ms	remaining: 43m 18s
5000:	learn: 0.0030513	total: 20m 31s	remaining: 20m 31s
9999:	learn: 0.0009042	total: 40m 53s	remaining: 0us
0:	learn: 0.0480293	total: 251ms	remaining: 41m 52s
5000:	learn: 0.0030769	total: 20m 7s	remaining: 20m 7s
9999:	learn: 0.0008837	total: 40m 9s	remaining: 0us
CPU times: user 21h 36s, sys: 9min 45s, total: 21h 10min 22s
Wall time: 5h 28min 31s


In [ ]:
mape = joblib.load(os.path.join('mape_catBA_0922.pickle'))
locals().update(mape)

mape

{'final_mape': [32.86395705835844],
 'test_mape': [33.14862431889832,
  32.71004211135829,
  33.00845864355376,
  32.56912996653439,
  32.88353025144744],
 'val_mape': [32.6931202322183,
  31.847535702539844,
  33.6770655001224,
  31.95436375476108,
  33.91874831091873]}

### catboost + Bayesian2
- 0923
- 처음 파라미터에서 범위를 좁힌다

In [ ]:
# Setting
X, y = X, log_y

# Setting parameter
'''
cb_basic = CatBoostRegressor(iterations=10000,
                             learning_rate=0.05,
                             depth=10,
                             eval_metric='MAPE',
                             random_seed = 77,
                             bagging_temperature = 0.2,
                             #group_id = []
                             od_type='Iter',
                             metric_period = 50,
                             od_wait=20,
                             thread_count=-1,
                             #task_type='gpu',
                             verbose=5000) 
'''
def catboost_eval(bagging_temperature ,    # 3h 30min
                  depth , 
                  learning_rate ,
                  min_data_in_leaf ,  
                  border_count):
    
    n_splits=5
    cv = KFold(n_splits=5, shuffle=True, random_state=77)
    mape = []
    predict = None

    params = {}
    params['iterations'] = 10000
    params['eval_metric'] = 'MAPE'
    params['random_seed'] = 77
    params['learning_rate'] = learning_rate
    params['min_data_in_leaf'] = int(round(min_data_in_leaf))
    params['depth'] = int(round(depth))
    #params['border_count'] = int(round(border_count))
    params['bagging_temperature'] = round(bagging_temperature)

    for tr_ind, val_ind in cv.split(X , y):
        X_train = X.iloc[tr_ind]
        y_train = y.iloc[tr_ind]
        X_valid = X.iloc[val_ind]
        y_valid = y.iloc[val_ind]

        # scaling
        scaler = MinMaxScaler()
        X_train = scaler.fit_transform(X_train)
        X_valid = scaler.transform(X_valid)
        
        model = CatBoostRegressor(**params , 
                                  leaf_estimation_iterations = 10,
                                  use_best_model=True,
                                  od_type="Iter",
                                  metric_period = 50,
                                  od_wait=20,
                                  #border_count=128,   # GPU 추천값
                                  thread_count=-1,
                                  task_type="GPU",
                                  verbose=5000
                                )
        model.fit(X_train, 
                  y_train,
                  eval_set=(X_valid, y_valid),
                  #iterations = 10000,
                  verbose = False
                  )
        
        y_pred = model.predict(X_valid)
        mape.append(MAPE_exp(y_pred,y_valid))
    return -np.mean(mape)

catBO2 = BayesianOptimization(catboost_eval,
                             {'bagging_temperature': (0.1, 0.3),
                              'depth': (8,12) ,
                              "learning_rate" : (0.4,0.6) , 
                              "min_data_in_leaf" : (1,6), 
                              'border_count': (125,130)    # GPU 추천값 사용
                             },
                             random_state=0)     # 77

catBO2.maximize(init_points=10, n_iter=20)    # 처음 10회 랜덤 값으로 score 계산 후 20회 최적화


|   iter    |  target   | baggin... | border... |   depth   | learni... | min_da... |
-------------------------------------------------------------------------------------
|  1        | -33.72    |  0.2098   |  128.6    |  10.41    |  0.509    |  3.118    |
|  2        | -33.88    |  0.2292   |  127.2    |  11.57    |  0.5927   |  2.917    |
|  3        | -33.86    |  0.2583   |  127.6    |  10.27    |  0.5851   |  1.355    |
|  4        | -33.98    |  0.1174   |  125.1    |  11.33    |  0.5556   |  5.35     |
|  5        | -33.89    |  0.2957   |  129.0    |  9.846    |  0.5561   |  1.591    |
|  6        | -33.57    |  0.228    |  125.7    |  11.78    |  0.5044   |  3.073    |
|  7        | -33.68    |  0.1529   |  128.9    |  9.825    |  0.5137   |  1.094    |
|  8        | -34.16    |  0.2235   |  128.1    |  10.47    |  0.5887   |  4.409    |
|  9        | -32.99    |  0.1719   |  127.2    |  10.79    |  0.412    |  4.334    |
|  10       | -33.35    |  0.2341   |  126.1    |  8.5

In [ ]:
# best parameter
best_params = catBO2.max

NameError: ignored

In [ ]:
# best catboost
cb_BO2 = CatBoostRegressor(
    **best_params,
    loss_function='MAPE',
    od_type="Iter",
    #eval_metric="AUC",
    task_type="GPU",
    iterations=10000,
    verbose=5000,
    )

TypeError: ignored

In [ ]:
# 평가 및 저장
mape_catBO2, pred_catBO2, SHAP_catBO2 = MAPE_exp_st(cb_OP, X, log_y, 5, 'catBO2_0923', scaling='MinMax') 

### CatBoost + OPTUNA  
- bagging_temperature': (0, 1000),
- depth': (5, 8.99) ,
- learning_rate" : (0.001,0.1) , 
- min_data_in_leaf" : (1,6) , 
- border_count': (5, 255) ,

In [ ]:
! pip install optuna
import optuna
optuna.logging.set_verbosity(optuna.logging.WARNING)

     |████████████████████████████████| 235kB 9.4MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 163kB 29.9MB/s 
     |████████████████████████████████| 81kB 10.4MB/s 
     |████████████████████████████████| 81kB 11.3MB/s 
     |████████████████████████████████| 112kB 31.0MB/s 
     |████████████████████████████████| 51kB 7.6MB/s 
     |████████████████████████████████| 133kB 30.6MB/s 
  Created wheel for optuna: filename=optuna-2.1.0-cp36-none-any.whl size=321090 sha256=b2c0046f8080d24df0027536e0d69862f0828a9a6a4928f8e54675cc41c37e3d
  Stored in directory: /root/.cache/pip/wheels/9f/25/24/a165483933b5eefbf4f93c85f3188dc696cbb38620b73ad713
Successfully built optuna
  Created wheel for pyperclip: filename=pyperclip-1.8.0-cp36-none-any.whl size=8693 sha256=8de1febe43963e79a68ad764f8b5ac0dd5f546b08e7db0d30b8d0b6f7139bcfa
  Stored in directory: /root/.cache/pip/wheels

In [ ]:
# 2번째 방법
# 13000초
def objective(trial, X, y):
    
    # 파라미터들의 search 범위
    params = {
        'depth': trial.suggest_int('depth', 5, 10), # (5, 8.99)
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 1,10),  
        'learning_rate': trial.suggest_uniform('learning_rate', 0.001, 0.1),
        'border_count': trial.suggest_int('border_count', 1, 260),
        'bagging_temperature': trial.suggest_int('bagging_temperature', 1, 1000)
    }

    # catboost setting
    cb_model = CatBoostRegressor(**params,
                                 verbose=5000,
                                 iterations=10000)
                                 #task_type = 'GPU')
    
    # cat boost : split + cv
    mape, pred, SHAP = MAPE_exp_st(cb_model, X, log_y, 5, 'catOP_0922', 'MinMax')
    return mape['final_mape'][0]

In [ ]:
%%time
cat_study = optuna.create_study()
cat_study.optimize(lambda x: objective(x, X, log_y), timeout=1000, n_jobs=-1)
print(cat_study.best_params, cat_study.best_value)

0:	learn: 0.9782445	total: 173ms	remaining: 28m 49s
0:	learn: 0.9834027	total: 164ms	remaining: 27m 17s
0:	learn: 0.9628285	total: 270ms	remaining: 44m 59s
0:	learn: 0.9706393	total: 1.57s	remaining: 4h 22m 1s
5000:	learn: 0.3107492	total: 14m 29s	remaining: 14m 29s
5000:	learn: 0.4481543	total: 12m 53s	remaining: 12m 53s
5000:	learn: 0.3289537	total: 13m 22s	remaining: 13m 22s
5000:	learn: 0.1806706	total: 20m 4s	remaining: 20m 4s
5000:	learn: 0.6010824	total: 20m 56s	remaining: 20m 56s
9999:	learn: 0.3859241	total: 25m 33s	remaining: 0us
0:	learn: 0.9803357	total: 140ms	remaining: 23m 23s
0:	learn: 0.9748310	total: 173ms	remaining: 28m 51s
5000:	learn: 0.1388829	total: 30m 25s	remaining: 30m 25s
9999:	learn: 0.2461376	total: 30m 30s	remaining: 0us
0:	learn: 0.9725738	total: 208ms	remaining: 34m 40s
5000:	learn: 0.0937604	total: 38m 1s	remaining: 38m
9999:	learn: 0.1095206	total: 40m 7s	remaining: 0us
0:	learn: 0.9594694	total: 241ms	remaining: 40m 13s
9999:	learn: 0.4869196	total: 44

In [ ]:
''' iter 10
{'depth': 10, 'min_data_in_leaf': 2, 'learning_rate': 0.09959582407712932, 'border_count': 162, 'bagging_temperature': 300} 82.67351335896514
CPU times: user 56min 9s, sys: 1min 49s, total: 57min 59s
Wall time: 18min 3s

iter 10000 optuna
{'depth': 9, 'min_data_in_leaf': 9, 'learning_rate': 0.04184730863160735, 'border_count': 254, 'bagging_temperature': 724} 32.54920644550847
CPU times: user 1d 7h 11min 18s, sys: 1h 10min, total: 1d 8h 21min 19s
Wall time: 8h 14min 29s
'''

### best OPTUNA + SHAP

In [ ]:
# best catboost
cb_OP = CatBoostRegressor(
    bagging_temperature = 724,
    border_count = 254,
    depth = 9,
    learning_rate = 0.0418,     #0.02017695398268462
    min_data_in_leaf = 9,
    leaf_estimation_iterations = 10,

    loss_function='MAPE',
    od_type="Iter",
    #eval_metric="AUC",
    task_type="GPU",
    iterations=10000,
    verbose=5000,
    )

In [ ]:
# 평가 및 저장
mape_catOP, pred_catOP, SHAP_catOP = MAPE_exp_st(cb_OP, X, log_y, 5, 'catOP_0923', scaling='MinMax') 

0:	learn: 0.0486718	total: 140ms	remaining: 23m 19s
5000:	learn: 0.0173731	total: 10m 27s	remaining: 10m 27s
9999:	learn: 0.0173335	total: 21m 32s	remaining: 0us
0:	learn: 0.0484897	total: 145ms	remaining: 24m 8s
5000:	learn: 0.0178524	total: 10m 32s	remaining: 10m 32s
9999:	learn: 0.0174555	total: 21m 20s	remaining: 0us
0:	learn: 0.0486812	total: 141ms	remaining: 23m 27s
5000:	learn: 0.0171103	total: 10m 38s	remaining: 10m 38s
9999:	learn: 0.0167694	total: 21m 42s	remaining: 0us
0:	learn: 0.0484890	total: 142ms	remaining: 23m 38s
5000:	learn: 0.0170224	total: 10m 35s	remaining: 10m 34s
9999:	learn: 0.0170191	total: 21m 40s	remaining: 0us
0:	learn: 0.0487019	total: 142ms	remaining: 23m 40s
5000:	learn: 0.0167105	total: 10m 40s	remaining: 10m 40s
9999:	learn: 0.0167083	total: 21m 56s	remaining: 0us


In [ ]:
mape_catOP

{'final_mape': [43.53248686607331],
 'test_mape': [43.129463818935015,
  43.82546879070623,
  43.19831405116548,
  43.4791251363258,
  44.03006253323403],
 'val_mape': [43.968674460215716,
  42.70422931013891,
  44.69485960629181,
  43.121629194566644,
  45.94196111345656]}

In [ ]:
pred_catOP

{'final_pred': [array([ 7413088.39832221, 12727260.57659936, 22983124.35989151, ...,
         35737533.57129582, 11789607.40227354, 11744679.19995264])],
 'test_idx': [Int64Index([ 8734,  7885,  2413, 12475,  4571,  9835, 35907, 14833, 10316,
              31140,
              ...
              25556, 30587,  6430, 11575, 33365, 14158, 28909, 34363, 26499,
               4615],
             dtype='int64', length=7076)],
 'test_pred': [array([ 6665621.12026682, 11116391.24175382, 23418053.79650783, ...,
         34239423.09384377, 11168197.63397961, 11529781.52197676]),
  array([ 8723745.45347949, 13303095.08577846, 21721053.44113622, ...,
         30887026.01685982, 12892111.65478494, 11940859.7481418 ]),
  array([ 7115374.96559032, 12377891.86657304, 22841662.95778179, ...,
         41726041.16752382, 11211702.91154813, 11563818.42069685]),
  array([ 7989923.7540691 , 11255202.4688839 , 22609200.82078594, ...,
         36740931.41946831, 12784883.39279641, 11886589.40889971]),
  array

In [ ]:
SHAP_catOP

{'expected_value': [16.57678559440221,
  16.580847828422325,
  16.573521488647387,
  16.57458837928757,
  16.573006929950395],
 'shap_values': [array([[ 3.24955205e-03,  6.50614652e-03,  8.88692040e-04, ...,
           2.99198010e-08,  0.00000000e+00,  0.00000000e+00],
         [ 3.29300762e-03,  2.40728087e-02,  4.62362031e-03, ...,
          -5.96825791e-08,  0.00000000e+00,  0.00000000e+00],
         [ 3.76115689e-03,  1.23691257e-02,  7.36154320e-03, ...,
          -1.91086406e-08,  0.00000000e+00,  0.00000000e+00],
         ...,
         [ 6.51605592e-03, -2.03767372e-03,  2.87143491e-03, ...,
           8.91275062e-08,  0.00000000e+00,  0.00000000e+00],
         [ 5.54913264e-03, -1.90719250e-04,  7.98829305e-03, ...,
          -1.02477888e-08,  0.00000000e+00,  0.00000000e+00],
         [ 6.31047173e-03, -1.05525341e-02, -1.60406989e-03, ...,
          -1.02477888e-08,  0.00000000e+00,  0.00000000e+00]]),
  array([[ 5.88900332e-04,  5.01066242e-03,  9.99606503e-03, ...,
        

### catboost + real default

In [ ]:
# best catboost
cb_de = CatBoostRegressor(
    loss_function='MAPE',
    od_type="Iter",
    task_type="GPU",
    iterations=10000,
    verbose=5000,
    )

In [ ]:
# 평가 및 저장
mape_catDE, pred_catDE, SHAP_catDE = MAPE_exp_st(cb_de, X, log_y, 5, 'catDE_0923', scaling='MinMax') 

0:	learn: 0.0489797	total: 17.4ms	remaining: 2m 53s
5000:	learn: 0.0368977	total: 1m 2s	remaining: 1m 2s
9999:	learn: 0.0337983	total: 2m 4s	remaining: 0us
0:	learn: 0.0488203	total: 17.7ms	remaining: 2m 57s
5000:	learn: 0.0362010	total: 1m 2s	remaining: 1m 2s
9999:	learn: 0.0331673	total: 2m 4s	remaining: 0us
0:	learn: 0.0489558	total: 20ms	remaining: 3m 20s
5000:	learn: 0.0361001	total: 1m 3s	remaining: 1m 3s
9999:	learn: 0.0325681	total: 2m 6s	remaining: 0us
0:	learn: 0.0487788	total: 17.2ms	remaining: 2m 52s
5000:	learn: 0.0371540	total: 1m 2s	remaining: 1m 2s
9999:	learn: 0.0328824	total: 2m 5s	remaining: 0us
0:	learn: 0.0489972	total: 20.6ms	remaining: 3m 26s
5000:	learn: 0.0381188	total: 1m 1s	remaining: 1m 1s
9999:	learn: 0.0339449	total: 2m 4s	remaining: 0us


In [ ]:
mape_catDE

{'final_mape': [81.16080589027379],
 'test_mape': [82.84773811705715,
  80.52428223227274,
  77.84615584764339,
  80.77015787662386,
  83.81569537777185],
 'val_mape': [86.85747938100327,
  81.43149229042814,
  78.1170823889536,
  84.66630188273186,
  87.83768276635334]}

In [ ]:
pred_catDE

{'final_pred': [array([ 9701724.97170959, 10912561.94373395, 20697539.53993221, ...,
         18598310.92802837,  9916746.56819834, 14316912.97888669])],
 'test_idx': [Int64Index([ 8734,  7885,  2413, 12475,  4571,  9835, 35907, 14833, 10316,
              31140,
              ...
              25556, 30587,  6430, 11575, 33365, 14158, 28909, 34363, 26499,
               4615],
             dtype='int64', length=7076)],
 'test_pred': [array([ 9766590.36587605, 10482525.49990246, 21989078.36119436, ...,
         17755155.96807333, 10291640.28476842, 15725894.72273028]),
  array([ 9840582.48818056, 11100838.38585444, 19510523.93587385, ...,
         18512081.24037703,  9792191.08559736, 14341182.81391367]),
  array([ 9478883.33950359, 11177797.89187157, 20528232.39303329, ...,
         20614723.68143107,  9491459.06617906, 14556138.40865984]),
  array([ 9053215.97059727, 10973010.60565952, 20856053.7697751 , ...,
         17149596.7598848 ,  9749381.11864405, 13207301.7574039 ]),
  array

In [ ]:
SHAP_catDE

{'expected_value': [16.605207325413815,
  16.600193187764333,
  16.592988251511276,
  16.604830327520236,
  16.604293462212436],
 'shap_values': [array([[ 8.85973086e-03, -5.84664359e-03, -1.43188423e-02, ...,
           5.77874370e-05,  6.31604620e-05, -1.20829018e-05],
         [ 8.47067832e-03, -7.63402490e-03, -1.45216935e-02, ...,
           5.77874370e-05,  4.36803686e-05, -1.20829018e-05],
         [ 9.01223324e-03, -3.17791587e-03, -1.11561144e-02, ...,
           3.82077383e-05,  3.13677759e-05, -3.02684543e-05],
         ...,
         [ 9.89888207e-03, -2.42432801e-03, -4.35435912e-03, ...,
           6.15482147e-05,  5.21895902e-05,  7.76470747e-05],
         [ 9.49664330e-03, -1.98162570e-03, -5.97123772e-03, ...,
           2.60316853e-05, -1.49082406e-04, -3.02684543e-05],
         [ 9.59168311e-03,  7.18396345e-03,  1.66410233e-02, ...,
           1.82722575e-05,  3.13677759e-05, -3.02684543e-05]]),
  array([[ 5.78757722e-03, -7.39232649e-03, -7.57188642e-03, ...,
      

# Catboost tunning
### 1. catBO
catboost + bayesian -> 학습시킬 때는 28, 하지만 cv 측정 50따리
### 2. catBA
catboost + basic -> cv 32  
### 3. catOP
catboost + OPTUNA -> cv 43
### 4. catDE
catboost + tunning x -> 81 따리...
### 5. catBO2
catboost + baysesian -> basic 범위와 비슷하게 학습 